# 04_extract_positions.ipynb


## Aufgabe

In diesem Notebook sollen Positionsinformationen (top/left) aus HTML-Elementen extrahiert werden.

Ziel: Extraktion und Analyse von Positionsdaten aus HTML-Elementen, um sie für weitere Verarbeitungsschritte oder Analysen zu verwenden.


## Bedienung

1. HTML-Code kann eingegeben werden:
   - als direkter HTML-String (in Textfeld)
   - oder per Dateipfad aus `data/original/`
2. Extraktion per Button starten
3. Positionsdaten werden als Tabelle angezeigt
4. Optional: Daten als CSV exportieren


## Technische Konstanten

- HTML_WIDTH = 1210
- HTML_HEIGHT = 825
- HTML_MARGIN_TOP = 0
- HTML_MARGIN_BOTTOM = 0


In [ ]:
# Benötigte Bibliotheken importieren
import sys
import os
import datetime
import pandas as pd
import json

# Pfad zum shared-Verzeichnis hinzufügen, falls es nicht im Pythonpath ist
sys.path.append('..')

# Importieren der gemeinsam genutzten Funktionen
from shared.html_utils import extract_positions, load_html_from_file, save_original_html
from shared.constants import HTML_HEIGHT, HTML_WIDTH


In [ ]:
# Installieren von benötigten Paketen, falls nicht vorhanden
try:
    import bs4
    print(f"BeautifulSoup Version: {bs4.__version__}")
except ImportError:
    print("BeautifulSoup ist nicht installiert. Installation wird gestartet...")
    !pip install beautifulsoup4
    import bs4
    print(f"BeautifulSoup Version: {bs4.__version__}")

try:
    import ipywidgets as widgets
    from IPython.display import display, HTML
    print(f"ipywidgets Version: {widgets.__version__}")
except ImportError:
    print("ipywidgets ist nicht installiert. Installation wird gestartet...")
    !pip install ipywidgets
    import ipywidgets as widgets
    from IPython.display import display, HTML
    print(f"ipywidgets Version: {widgets.__version__}")

try:
    import pandas as pd
    print(f"Pandas Version: {pd.__version__}")
except ImportError:
    print("Pandas ist nicht installiert. Installation wird gestartet...")
    !pip install pandas
    import pandas as pd
    print(f"Pandas Version: {pd.__version__}")


In [ ]:
# Beispiel-HTML für die Demonstration
example_html = """<!DOCTYPE html>
<html>
<head>
    <meta http-equiv="X-UA-Compatible" content="IE=Edge" />
    <meta charset="utf-8" />
</head>

<body style="margin: 0;">

<div id="p1" style="overflow: hidden; position: relative; background-color: white; width: 1210px; height: 825px;">

    <!-- Begin shared CSS values -->
    <style class="shared-css" type="text/css" >
        .t {
            transform-origin: bottom left;
            z-index: 2;
            position: absolute;
            white-space: pre;
            overflow: visible;
            line-height: 1.5;
        }
        .text-container {
            white-space: pre;
        }
        @supports (-webkit-touch-callout: none) {
            .text-container {
                white-space: normal;
            }
        }
    </style>
    <!-- End shared CSS values -->

    <!-- Begin inline CSS -->
    <style type="text/css" >
        #t1_1{left:18px;top:21px;letter-spacing:0.14px;}
        #t2_1{left:128px;top:21px;}
        #t3_1{left:165px;top:21px;letter-spacing:0.15px;}
        #t4_1{left:18px;bottom:804px;letter-spacing:0.14px;}
        #t5_1{left:128px;bottom:804px;}
        #t6_1{left:165px;bottom:804px;letter-spacing:0.15px;}
    </style>
    <!-- End inline CSS -->

    <!-- Begin text -->
    <div id="t1_1" class="t">Beispieltext 1 (top)</div>
    <div id="t2_1" class="t">Beispieltext 2 (top)</div>
    <div id="t3_1" class="t">Beispieltext 3 (top)</div>
    <div id="t4_1" class="t">Beispieltext 4 (bottom)</div>
    <div id="t5_1" class="t">Beispieltext 5 (bottom)</div>
    <div id="t6_1" class="t">Beispieltext 6 (bottom)</div>
    <!-- End text -->

</div>
</body>
</html>"""


In [ ]:
# Erstellen der Widgets für die Benutzeroberfläche
html_input = widgets.Textarea(
    value=example_html,
    placeholder='HTML-Code hier einfügen',
    description='HTML-Code:',
    disabled=False,
    layout=widgets.Layout(width='100%', height='200px')
)

file_input = widgets.Text(
    value='',
    placeholder='Dateipfad eingeben (z.B. original_2025-07-16_104156.html)',
    description='Datei:',
    disabled=False,
    layout=widgets.Layout(width='80%')
)

extract_button = widgets.Button(
    description='Positionen extrahieren',
    disabled=False,
    button_style='success',
    tooltip='Klicken, um Positionen zu extrahieren',
    icon='check'
)

export_button = widgets.Button(
    description='Als CSV exportieren',
    disabled=True,
    button_style='info',
    tooltip='Klicken, um Positionen als CSV zu exportieren',
    icon='download'
)

output = widgets.Output()

# Anzeigen der Widgets
print("HTML-Code eingeben oder Datei auswählen:")
display(html_input)
print("\nODER Datei aus data/original/ auswählen:")
display(file_input)
display(extract_button)
display(export_button)
display(output)


In [ ]:
# Globale Variable für extrahierte Positionen
extracted_positions = []

# Funktion für den Extraktionsbutton
def on_extract_button_clicked(b):
    global extracted_positions
    
    with output:
        output.clear_output()
        
        # HTML-Code aus Textfeld oder Datei laden
        html_string = ""
        if file_input.value:
            file_path = os.path.join('..', 'data', 'original', file_input.value)
            print(f"Lade HTML aus Datei: {file_path}")
            html_string = load_html_from_file(file_path)
            if not html_string:
                print("Fehler: Datei konnte nicht geladen werden.")
                return
        else:
            html_string = html_input.value
            if not html_string.strip():
                print("Fehler: Kein HTML-Code eingegeben.")
                return
            
            # Original-HTML speichern
            timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H%M%S")
            original_path = save_original_html(html_string, timestamp)
            print(f"Original-HTML gespeichert unter: {original_path}")
        
        print("Extrahiere Positionsdaten aus HTML...")
        
        # Positionen extrahieren
        extracted_positions = extract_positions(html_string)
        
        if not extracted_positions:
            print("Keine Positionsdaten gefunden.")
            export_button.disabled = True
            return
        
        # Positionen in DataFrame konvertieren für bessere Darstellung
        positions_df = pd.DataFrame(extracted_positions)
        
        # Fehlende Werte mit "N/A" füllen
        positions_df = positions_df.fillna("N/A")
        
        # Berechnen der top-Werte für Elemente mit bottom-Werten
        for i, row in positions_df.iterrows():
            if row['bottom'] != "N/A" and row['top'] == "N/A":
                bottom_value = float(row['bottom'])
                top_value = HTML_HEIGHT - bottom_value
                positions_df.at[i, 'calculated_top'] = top_value
            else:
                positions_df.at[i, 'calculated_top'] = "N/A"
        
        # Anzeigen der Ergebnisse
        print(f"\nGefundene Positionsdaten ({len(extracted_positions)} Elemente):")
        display(positions_df)
        
        # Export-Button aktivieren
        export_button.disabled = False
        
        # Visualisierung der Positionen
        print("\nVisualisierung der Positionen:")
        
        # Einfache HTML-Visualisierung erstellen
        visualization_html = """
        <div style="position: relative; width: 1210px; height: 825px; border: 1px solid black; background-color: #f0f0f0;">
        """
        
        for pos in extracted_positions:
            element_id = pos['id']
            
            # Position bestimmen (entweder direkt top oder berechnet aus bottom)
            if 'top' in pos and pos['top'] != "N/A":
                top = pos['top']
            elif 'bottom' in pos and pos['bottom'] != "N/A":
                top = HTML_HEIGHT - float(pos['bottom'])
            else:
                continue
                
            left = pos.get('left', 0)
            
            # Element zur Visualisierung hinzufügen
            visualization_html += f"""
            <div style="position: absolute; left: {left}px; top: {top}px; 
                        background-color: rgba(0, 128, 255, 0.3); 
                        border: 1px solid blue; padding: 2px; 
                        font-family: Arial; font-size: 10px;">
                {element_id}
            </div>
            """
        
        visualization_html += "</div>"
        
        # Visualisierung anzeigen
        display(HTML(visualization_html))

# Funktion für den Export-Button
def on_export_button_clicked(b):
    global extracted_positions
    
    with output:
        if not extracted_positions:
            print("Keine Daten zum Exportieren vorhanden.")
            return
        
        # DataFrame erstellen
        positions_df = pd.DataFrame(extracted_positions)
        
        # Fehlende Werte mit "N/A" füllen
        positions_df = positions_df.fillna("N/A")
        
        # Berechnen der top-Werte für Elemente mit bottom-Werten
        for i, row in positions_df.iterrows():
            if row.get('bottom', "N/A") != "N/A" and row.get('top', "N/A") == "N/A":
                bottom_value = float(row['bottom'])
                top_value = HTML_HEIGHT - bottom_value
                positions_df.at[i, 'calculated_top'] = top_value
        
        # CSV-Datei erstellen
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H%M%S")
        csv_filename = f"extracted_positions_{timestamp}.csv"
        csv_path = os.path.join('..', 'data', 'output', csv_filename)
        
        # Sicherstellen, dass der Ausgabeordner existiert
        os.makedirs(os.path.dirname(csv_path), exist_ok=True)
        
        # CSV speichern
        positions_df.to_csv(csv_path, index=False)
        
        print(f"Positionsdaten wurden exportiert nach: {csv_path}")

# Button-Klick-Events registrieren
extract_button.on_click(on_extract_button_clicked)
export_button.on_click(on_export_button_clicked)


## Erklärung der Positionsextraktion

Die Extraktion von Positionsdaten aus HTML-Elementen ist nützlich für:

1. **Analyse der Elementverteilung**: Verstehen, wie Elemente auf der Seite angeordnet sind
2. **Vorbereitung für Konvertierungen**: Identifizieren von Elementen, die Anpassungen benötigen
3. **Debugging**: Überprüfen, ob Positionsänderungen korrekt angewendet wurden

Die extrahierten Daten enthalten:
- **Element-ID**: Eindeutige Kennung des Elements
- **Left-Position**: Horizontale Position von links
- **Top-Position**: Vertikale Position von oben (wenn vorhanden)
- **Bottom-Position**: Vertikale Position von unten (wenn vorhanden)
- **Berechnete Top-Position**: Für Elemente mit bottom-Werten wird die entsprechende top-Position berechnet

Diese Daten können als CSV exportiert werden für:
- Weiterverarbeitung in anderen Tools (Excel, Datenanalyse-Software)
- Dokumentation der ursprünglichen Positionen vor Konvertierungen
- Vergleich von Positionen vor und nach Anpassungen